### Import required libraries 

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


### load dataset

In [88]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r"E:\reviews_data_dump\reviews_badminton\data.csv")
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You.READ MORE",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside cover was Yonex Ad inside was a cheapest.... Sad to hear this.READ MORE,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in new box. It's not a original yonex product. Don't buy.flipkart platform is chosen to fraud the buyers.READ MORE,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the corks like not as before 3 to 5 years back.. I am using MAVIS 350 for more than 15 years quality of corks was very very good at that times, but now I am not getting the quality corks as like before, rate of corks also too much now, I am very sorry to say like this, but in my experience , my Statment is very true to my knowledgeREAD MORE",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn't understand.. Wat is d advantage of buying dis frm flipkrtREAD MORE,1


In [89]:
df = df.rename(columns={
    "Reviewer Name": "reviewer_name",
    "Review Title": "review_title",
    "Place of Review": "review_location",
    "Up Votes": "upvotes",
    "Down Votes": "downvotes",
    "Month": "review_month",
    "Review text": "review_text",
    "Ratings": "rating"
})


In [90]:
df = df.drop(columns=["reviewer_name"])


In [91]:
df.columns

Index(['review_title', 'review_location', 'upvotes', 'downvotes',
       'review_month', 'review_text', 'rating'],
      dtype='object')

In [92]:
df["review_location"] = (
    df["review_location"]
    .astype(str)
    .str.replace("Certified Buyer", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

In [93]:
df["review_month"].unique()

array(['Feb 2021', 'Apr 2021', nan, 'Apr 2016', 'Oct 2018', 'Jan 2020',
       'Aug 2018', 'May 2018', 'Jun 2020', 'Sep 2016', 'Jun 2018',
       'Mar 2021', 'Mar 2019', 'Oct 2016', 'Feb 2018', 'May 2021',
       'Aug 2020', 'Feb 2019', 'May 2020', 'Jul 2018', 'Nov 2020',
       'Mar 2020', 'Jan 2022', 'Jan 2021', 'May 2017', 'Nov 2018',
       'Dec 2018', 'Oct 2022', 'Sep 2022', 'Nov 2021', 'Oct 2021',
       'Aug 2021', 'Jun 2021', 'Sep 2020', 'Jul 2020', 'Dec 2019',
       'Sep 2019', 'Jun 2019', 'Apr 2019', 'Jan 2019', 'Sep 2017',
       'Jan 2018', 'Dec 2017', 'Oct 2017', 'Apr 2018', 'Dec 2021',
       'Dec 2020', 'Mar 2018', 'Jul 2017', 'May 2019', 'Nov 2019',
       'Jul 2019', 'Nov 2022', 'Jul 2022', 'May 2022', 'Mar 2022',
       'Feb 2022', 'Jul 2021', 'Feb 2020', 'Aug 2019', 'Sep 2018',
       'Nov 2017', 'Feb 2017', 'Jun 2017', 'Aug 2016', 'Dec 2016',
       'Mar 2017', 'Jan 2017', 'Aug 2017', 'Aug 2022', 'Jun 2022',
       'Apr 2022', 'Sep 2021', 'Oct 2020', 'Apr 2020', 'O

In [94]:
df["review_date"] = pd.to_datetime(
    df["review_month"],
    format="%b %Y",
    errors="coerce"
)


In [95]:
df["review_year"] = df["review_date"].dt.year
df["review_month_num"] = df["review_date"].dt.month


In [96]:
df.head()

,review_title,review_location,upvotes,downvotes,review_month,review_text,rating,review_date,review_year,review_month_num
0,Nice product,Chirakkal,889.0,64.0,Feb 2021,"Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You.READ MORE",4,2021-02-01,2021.0,2.0
1,Don't waste your money,Hyderabad,109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside cover was Yonex Ad inside was a cheapest.... Sad to hear this.READ MORE,1,2021-02-01,2021.0,2.0
2,Did not meet expectations,Dharmapuri,42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in new box. It's not a original yonex product. Don't buy.flipkart platform is chosen to fraud the buyers.READ MORE,1,2021-04-01,2021.0,4.0
3,Fair,Chennai,25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the corks like not as before 3 to 5 years back.. I am using MAVIS 350 for more than 15 years quality of corks was very very good at that times, but now I am not getting the quality corks as like before, rate of corks also too much now, I am very sorry to say like this, but in my experience , my Statment is very true to my knowledgeREAD MORE",3,NaT,NaN,NaN
4,Over priced,nan,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn't understand.. Wat is d advantage of buying dis frm flipkrtREAD MORE,1,2016-04-01,2016.0,4.0


In [97]:
df = df.drop(columns=["review_month","review_date"])

In [98]:
df.head()

,review_title,review_location,upvotes,downvotes,review_text,rating,review_year,review_month_num
0,Nice product,Chirakkal,889.0,64.0,"Nice product, good quality, but price is now rising which is a bad sign. 800-850 was an affordable price, especially when we play everyday. So kindly help us out in terms of the price. Thank You.READ MORE",4,2021.0,2.0
1,Don't waste your money,Hyderabad,109.0,6.0,They didn't supplied Yonex Mavis 350. Outside cover was Yonex Ad inside was a cheapest.... Sad to hear this.READ MORE,1,2021.0,2.0
2,Did not meet expectations,Dharmapuri,42.0,3.0,Worst product. Damaged shuttlecocks packed in new box. It's not a original yonex product. Don't buy.flipkart platform is chosen to fraud the buyers.READ MORE,1,2021.0,4.0
3,Fair,Chennai,25.0,1.0,"Quite O. K. , but nowadays the quality of the corks like not as before 3 to 5 years back.. I am using MAVIS 350 for more than 15 years quality of corks was very very good at that times, but now I am not getting the quality corks as like before, rate of corks also too much now, I am very sorry to say like this, but in my experience , my Statment is very true to my knowledgeREAD MORE",3,NaN,NaN
4,Over priced,nan,147.0,24.0,Over pricedJust â?¹620 ..from retailer.I didn't understand.. Wat is d advantage of buying dis frm flipkrtREAD MORE,1,2016.0,4.0


### handling missing values

In [99]:
df = df.dropna(subset=["review_text"])


In [100]:
df["rating"] = df["rating"].fillna(df["rating"].median())


In [101]:
df = df.dropna(subset=["review_year"])


In [102]:
df = df.dropna(subset=["review_month_num"])

### change data types

In [103]:
df["upvotes"] = df["upvotes"].fillna(0).astype(int)


In [104]:
df["downvotes"] = df["downvotes"].fillna(0).astype(int)


In [105]:
df.isna().sum()

review_title        0
review_location     0
upvotes             0
downvotes           0
review_text         0
rating              0
review_year         0
review_month_num    0
dtype: int64

### Data preprocessing

In [106]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [107]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [108]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

df["clean_review_text"] = df["review_text"].apply(preprocess)

In [73]:
### BoW

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
bow_features = bow.fit_transform(df["clean_review_text"])

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_features = tfidf.fit_transform(df["clean_review_text"])

In [111]:
df["sentiment"] = df["rating"].apply(lambda x: 1 if x >= 4 else 0)

In [113]:
df.drop(columns=['review_text'])

,review_title,review_location,upvotes,downvotes,rating,review_year,review_month_num,clean_review_text,sentiment
0,Nice product,Chirakkal,889,64,4,2021.0,2.0,nice product good quality price rising bad sign affordable price especially play everyday kindly help u term price thank youread,1
1,Don't waste your money,Hyderabad,109,6,1,2021.0,2.0,didnt supplied yonex mavis outside cover yonex ad inside cheapest sad hear thisread,0
2,Did not meet expectations,Dharmapuri,42,3,1,2021.0,4.0,worst product damaged shuttlecock packed new box original yonex product dont buyflipkart platform chosen fraud buyersread,0
4,Over priced,nan,147,24,1,2016.0,4.0,pricedjust retaileri didnt understand wat advantage buying dis frm flipkrtread,0
5,Mind-blowing purchase,Hyderabad,173,45,5,2018.0,10.0,good quality product delivered timeread,1
...,...,...,...,...,...,...,...,...,...
8503,Yones Mavis 350 Blue cap,nan,2,0,1,2016.0,4.0,wrost duplicate productdont buy seller productflipkart ko seller ko block kr dena chahiyeread,0
8504,For Mavis350,nan,2,1,5,2016.0,8.0,received product intact sealedread,1
8505,Very Good,nan,1,0,3,2016.0,9.0,delivered time price high marketread,0
8506,Don't waste your money,nan,0,2,4,2016.0,9.0,mark available market less priceread,1


In [126]:
final_df1 = df[["clean_review_text", "sentiment"]]


In [127]:
final_df1.head()

,clean_review_text,sentiment
0,nice product good quality price rising bad sign affordable price especially play everyday kindly help u term price thank youread,1
1,didnt supplied yonex mavis outside cover yonex ad inside cheapest sad hear thisread,0
2,worst product damaged shuttlecock packed new box original yonex product dont buyflipkart platform chosen fraud buyersread,0
4,pricedjust retaileri didnt understand wat advantage buying dis frm flipkrtread,0
5,good quality product delivered timeread,1


In [129]:
final_df1.to_csv(r'E:\final_df1.csv')

### Train-test split

In [77]:
from sklearn.model_selection import train_test_split

X = df["clean_review_text"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)

print("Logistic Regression F1 Score:", f1)
print("Logistic Regression Accuracy",accuracy)
print("Logistic Regression Precision",precision)
print("Logistic Regression Recall",recall)

Logistic Regression F1 Score: 0.9216757741347905
Logistic Regression Accuracy 0.8665425201738051
Logistic Regression Precision 0.8736187845303868
Logistic Regression Recall 0.9753276792598303


### Evaluation

In [80]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [85]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Linear SVM": LinearSVC()
}

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    preds = model.predict(X_test_tfidf)
    print(name,"Accuracy",accuracy_score(y_test,preds))
    print(name,"Precision",precision_score(y_test,preds))
    print(name,"Recall",recall_score(y_test,preds))
    print(name, "F1:", f1_score(y_test, preds))


Logistic Regression Accuracy 0.8665425201738051
Logistic Regression Precision 0.8736187845303868
Logistic Regression Recall 0.9753276792598303
Logistic Regression F1: 0.9216757741347905
Naive Bayes Accuracy 0.8584729981378026
Naive Bayes Precision 0.859448554135844
Naive Bayes Recall 0.9853508095605242
Naive Bayes F1: 0.9181034482758621
Linear SVM Accuracy 0.8702669149596524
Linear SVM Precision 0.8885714285714286
Linear SVM Recall 0.9591364687740941
Linear SVM F1: 0.9225064886911383


### Conclusions 
- Among all evaluated models, Linear SVM achieved the highest F1-score, indicating the best balance between precision and recall. Therefore, Linear SVM was selected as the final model for sentiment classification.